# Contest Parser v1
Goal is to return the following table
 
handle | rating | contestID | problemID | success 
--- | --- | --- | --- | --- | ---
handle0 | 1234 | 633 | A | 1
handle0 | 1234 | 633 | B | 1
handle0 | 1234 | 633 | C | 0


In [164]:
import requests
import pandas as pd
from collections import defaultdict

In [165]:
contestID = 398
urlbase = 'http://codeforces.com/api/'
method = 'contest.standings'
maxcnt = 100000
#http://codeforces.com/api/contest.standings?contestId=566&from=1&count=5&showUnofficial=true
url = urlbase + method + '?contestId=' + str(contestID) + '&from=1&count=' + str(maxcnt) + '&showUnofficial=false'
r = requests.get(url).json()['result']
rows = r['rows']
problems = r['problems']
contest = r['contest']

## row objects
**keys:**
```
- successfulHackCount
- unsuccessfulHackCount
- rank
- penalty
- points
- party
- problemResults
```

Every entry appears to have the same keys
```python
test = rows[0].keys()
all([r.keys() == test for r in rows])
```

**problem results**
```
bestSubmissionTimeSeconds': 3624 <-- this field is absent if the user has points 0
points': 1330.0
rejectedAttemptCount': 0
type': u'FINAL' <-- All should be FINAL
```

```python
test = []
for r in rows:
    for rr in r['problemResults']:
        test.append(rr['type'])
all([t == 'FINAL' for t in test])
```
returns True

The following does not print anything:
```python
test = rows[0]['problemResults'][0].keys()
for r in rows:
    if not r['problemResults'][0].keys() == test and r['problemResults'][0]['points']!=0:
        print r['problemResults'][0]
```

In [161]:
# CHECK TO MAKE SURE THAT TEAMS ARE NOT ALLOWED!!!
for r in rows: # for each person
    if len(r['party']['members']) > 1:
        print "------------------NEED TO THROW ERROR HERE, CONTEST ALLOWS TEAMS"
        break

In [162]:
users = []
points = []
#ranks = []
for r in rows:
    users.append(r['party']['members'][0]['handle'])
    
    #ranks.append(r['rank'])
    
    userpts = [0]*len(problems)
    for i in range(len(problems)):
        userpts[i] = r['problemResults'][i]['points']
    points.append([1 if u > 0 else 0 for u in userpts])

[]

# Rating change parser


In [153]:
# Grab rating changes
method = 'contest.ratingChanges'
url = urlbase + method + '?contestId=' + str(contestID)
ratingchanges = requests.get(url).json()['result']
ratingdict = dict()
for r in ratingchanges:
    ratingdict[r['handle']] = r['oldRating']

In [154]:
url

'http://codeforces.com/api/contest.ratingChanges?contestId=398'

# Create output df 

In [155]:
# start constructing dataframe
array_out = []
for i in range(len(users)): # for each user in the contest
    hdl = users[i]
    print "handle:", hdl
    rating = ratingdict[hdl]
    for j, p in enumerate(problems): # for each problem in the contest, make a new row
        temp = dict.fromkeys(['handle', 'rating', 'contestID', 'problemID', 'success'])
        temp['handle'] = hdl
        temp['contestID'] = contestID
        temp['problemID'] = p['index']
        temp['success'] = points[i][j]
        temp['rating'] = rating
        array_out.append(temp)
    

In [156]:
output = pd.DataFrame.from_dict(array_out)

0